# Part 1: LLM – Exploratory Data Analysis & Missing Data Imputation

In **Part 1**, we begin our Final Project into the SAP product dataset provided by Kärcher. Our goal is to prepare and enrich this data for downstream interactions with a large language model (LLM). The steps in this phase are:

1. **Classical Exploratory Data Analysis (EDA)**
   - Load and inspect the raw dataset extract from SAP.
   - Examine each field’s data type, distribution, and cardinality.
   - Compute basic descriptive statistics to gain a holistic view of the dataset.

2. **Data Transformation for LLM Consumption**
   - Convert original “object”-typed fields into appropriate **categorical** or **numerical** types.
   - Document all transformation rules and mappings to ensure reproducibility.

3. **Missing Data Identification & Imputation Strategy**
   - Identify records and cells with missing values in critical fields.
   - Leverage the **product description PDFs** (extracted in Part 0 using agentic embeddings) as our external knowledge source.
   - Query the IBM Granite 13B Instruct V2 model to cross-reference the markdowns from Part 0 and suggest imputed values for each missing field.

4. **Validation & Iteration**
   - Cross-check the LLM’s imputations against ground-truth data or manual inspection of the source PDFs.


In [1]:
import sys
print(sys.version)

3.10.17 | packaged by conda-forge | (main, Apr 10 2025, 22:06:35) [MSC v.1943 64 bit (AMD64)]


In [2]:
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from tqdm import tqdm
import difflib 
import json
import re

In [3]:
project_root = Path.cwd()  
data_file    = project_root / "data" / "SAP_Produktstammdaten_vfinal.csv"

df = pd.read_csv(
    data_file,
    sep=",",                # comma-separated
    engine="python",        # tolerant parser
    quotechar='"',          # respect commas inside quotes
    skipinitialspace=True,  # trim spaces after delimiters
    dtype=str               # load all as strings initially
)

original = df.copy(deep=True)

print(df.shape)
df.head()


(68, 17)


,Produktname,Bestellnummer,Preis (€ inkl. MwSt.),Lieferzeit,Stromart (V/Hz),Druck (bar/MPa),Fördermenge (l/h),Flächenleistung (m²/h),Zulauftemperatur (°C),Anschlussleistung (kW),Anschlusskabel (m),Farbe,Gewicht ohne Zubehör (kg),Gewicht inkl. Verpackung (kg),Abmessungen (L × B × H) (mm),Lieferumfang,Ausstattung
0,K 7 Premium Smart Control Flex eco!B,1.317-365.0,664.99,2-3 Werktage,230 / 50,20 - max. 180 / 2 - max. 18,max. 600,60,max. 60,3,5,gelb,17.8,24.4,459 x 330 x 669,"G 180 Q Smart Control, Multi Jet 3-in-1, eco!B...","Teleskopgriff, Integrierter Wasserfilter"
1,K 7 Premium Smart Control Flex Home,1.317-363.0,694.99,2-3 Werktage,230 / 50,20 - max. 180 / 2 - max. 18,max. 600,60,max. 60,3,5,gelb,17.8,25.6,459 x 330 x 669,"Home Kit: Flächenreiniger T 7, Stein- und Fass...","Integrierte HD-Schlauchtrommel, Integriertes A..."
2,K 7 Premium Power Flex,1.317-320.0,554.99,2-3 Werktage,230 / 50,20 - max. 180 / 2 - max. 18,max. 600,60,max. 60,3,5,gelb,17.8,22.2,459 x 330 x 669,"G 180 Q, Dreckfräser, HD-Schlauch 10 m Premium...","Vario Power Jet, Integrierte HD-Schlauchtromme..."
3,K 7 Premium Power Flex Home,1.317-322.0,624.99,2-3 Werktage,230 / 50,20 - max. 180 / 2 - max. 18,max. 600,60,max. 60,3,5,gelb,17.8,25.4,459 x 330 x 669,"Home Kit: Flächenreiniger T 7, Stein- und Fass...","Vario Power Jet, Integrierte HD-Schlauchtromme..."
4,K 7 Premium Smart Control Flex,1.317-360.0,624.99,2-3 Werktage,230 / 50,20 - max. 180 / 2 - max. 18,max. 600,60,max. 60,3,5,gelb,17.8,23,459 x 330 x 669,"G 180 Q Smart Control, Multi Jet 3-in-1, HD-Sc...","Integrierte HD-Schlauchtrommel, Integriertes A..."


In [4]:
print("Shape:", df.shape)
print("\nColumn types:")
print(df.dtypes)

Shape: (68, 17)

Column types:
Produktname                      object
Bestellnummer                    object
Preis (€ inkl. MwSt.)            object
Lieferzeit                       object
Stromart (V/Hz)                  object
Druck (bar/MPa)                  object
Fördermenge (l/h)                object
Flächenleistung (m²/h)           object
Zulauftemperatur (°C)            object
Anschlussleistung (kW)           object
Anschlusskabel (m)               object
Farbe                            object
Gewicht ohne Zubehör (kg)        object
Gewicht inkl. Verpackung (kg)    object
Abmessungen (L × B × H) (mm)     object
Lieferumfang                     object
Ausstattung                      object
dtype: object


In order to proceed with data manipulation we would need to change the types of the data in order to be able to perfom manipulations that would not be possible if numbers are stored as string value.

In [5]:
numeric_cols = [
    "Preis (€ inkl. MwSt.)",
    "Flächenleistung (m²/h)",
    "Anschlussleistung (kW)",
    "Anschlusskabel (m)",
    "Gewicht ohne Zubehör (kg)",
    "Gewicht inkl. Verpackung (kg)"
]

for col in numeric_cols:
    # German‐style decimals: comma → dot
    cleaned = (
        df[col]
        .str.replace(r"[^\d,.\-]", "", regex=True)
        .str.replace(",", ".", regex=False)
    )
    df[col] = pd.to_numeric(cleaned, errors="coerce")

cat_cols = [c for c in df.columns if c not in numeric_cols]
df[cat_cols] = df[cat_cols].astype("category")

print(df.dtypes)
print(df[numeric_cols].describe())


Produktname                      category
Bestellnummer                    category
Preis (€ inkl. MwSt.)             float64
Lieferzeit                       category
Stromart (V/Hz)                  category
Druck (bar/MPa)                  category
Fördermenge (l/h)                category
Flächenleistung (m²/h)            float64
Zulauftemperatur (°C)            category
Anschlussleistung (kW)            float64
Anschlusskabel (m)                float64
Farbe                            category
Gewicht ohne Zubehör (kg)         float64
Gewicht inkl. Verpackung (kg)     float64
Abmessungen (L × B × H) (mm)     category
Lieferumfang                     category
Ausstattung                      category
dtype: object
       Preis (€ inkl. MwSt.)  Flächenleistung (m²/h)  Anschlussleistung (kW)  \
count              60.000000               66.000000               65.000000   
mean              352.948333               37.045455                2.066154   
std               184.277961    

In [6]:
print("\nMissing values per column:")
print(df.isna().sum())


Missing values per column:
Produktname                      0
Bestellnummer                    0
Preis (€ inkl. MwSt.)            8
Lieferzeit                       0
Stromart (V/Hz)                  0
Druck (bar/MPa)                  0
Fördermenge (l/h)                0
Flächenleistung (m²/h)           2
Zulauftemperatur (°C)            0
Anschlussleistung (kW)           3
Anschlusskabel (m)               7
Farbe                            0
Gewicht ohne Zubehör (kg)        0
Gewicht inkl. Verpackung (kg)    0
Abmessungen (L × B × H) (mm)     0
Lieferumfang                     0
Ausstattung                      0
dtype: int64


As we can see, some of the data is already missing, lets increase the number of missing data by blanking 10%, this will give us the possibility to check the accuracy of the LLM performance at the end as we know the "banked" values.

In [7]:
df_test = df.copy(deep=True)

n_rows, n_cols = df_test.shape
total_cells     = n_rows * n_cols
n_blanks        = int(total_cells * 0.1)


product_col = df_test.columns.get_loc("Produktname")

# Builds a list of all flat indices except those in Produktname
# For each row r and column c ≠ product_col, flat_index = r*n_cols + c
valid_indices = [
    r * n_cols + c
    for r in range(n_rows)
    for c in range(n_cols)
    if c != product_col
]

# Picks 10% of those valid positions at random (reproducibly)
rng = np.random.default_rng(seed=42)
flat_indices = rng.choice(valid_indices, size=n_blanks, replace=False)

# Maps flat indices back to (row, col) and set to NaN
for idx in flat_indices:
    i = idx // n_cols      # row index
    j = idx %  n_cols      # col index
    df_test.iat[i, j] = np.nan

# Save to a new CSV (ensure your `data/processed` folder exists)
out_path = project_root / "data" / "processed" / "SAP_Produktstammdaten_vfinal_test_missing.csv"
out_path.parent.mkdir(parents=True, exist_ok=True)
df_test.to_csv(out_path, index=False)

print(f"Test CSV with 10% random blanks written to: {out_path}")


Test CSV with 10% random blanks written to: c:\Users\charl\Documents\CBS\Code Projects_Python\AIML25\AIML25_Project\data\processed\SAP_Produktstammdaten_vfinal_test_missing.csv


In [8]:
project_root = Path.cwd()  
data_file    = project_root / "data" / "processed" / "SAP_Produktstammdaten_vfinal_test_missing.csv"
df_test = pd.read_csv(
    data_file,
    sep=",",                
    engine="python",        
    quotechar='"',          
    skipinitialspace=True,  
    dtype=str               
)


print(df.shape)
df.head()


(68, 17)


,Produktname,Bestellnummer,Preis (€ inkl. MwSt.),Lieferzeit,Stromart (V/Hz),Druck (bar/MPa),Fördermenge (l/h),Flächenleistung (m²/h),Zulauftemperatur (°C),Anschlussleistung (kW),Anschlusskabel (m),Farbe,Gewicht ohne Zubehör (kg),Gewicht inkl. Verpackung (kg),Abmessungen (L × B × H) (mm),Lieferumfang,Ausstattung
0,K 7 Premium Smart Control Flex eco!B,1.317-365.0,664.99,2-3 Werktage,230 / 50,20 - max. 180 / 2 - max. 18,max. 600,60.0,max. 60,3.0,5.0,gelb,17.8,24.4,459 x 330 x 669,"G 180 Q Smart Control, Multi Jet 3-in-1, eco!B...","Teleskopgriff, Integrierter Wasserfilter"
1,K 7 Premium Smart Control Flex Home,1.317-363.0,694.99,2-3 Werktage,230 / 50,20 - max. 180 / 2 - max. 18,max. 600,60.0,max. 60,3.0,5.0,gelb,17.8,25.6,459 x 330 x 669,"Home Kit: Flächenreiniger T 7, Stein- und Fass...","Integrierte HD-Schlauchtrommel, Integriertes A..."
2,K 7 Premium Power Flex,1.317-320.0,554.99,2-3 Werktage,230 / 50,20 - max. 180 / 2 - max. 18,max. 600,60.0,max. 60,3.0,5.0,gelb,17.8,22.2,459 x 330 x 669,"G 180 Q, Dreckfräser, HD-Schlauch 10 m Premium...","Vario Power Jet, Integrierte HD-Schlauchtromme..."
3,K 7 Premium Power Flex Home,1.317-322.0,624.99,2-3 Werktage,230 / 50,20 - max. 180 / 2 - max. 18,max. 600,60.0,max. 60,3.0,5.0,gelb,17.8,25.4,459 x 330 x 669,"Home Kit: Flächenreiniger T 7, Stein- und Fass...","Vario Power Jet, Integrierte HD-Schlauchtromme..."
4,K 7 Premium Smart Control Flex,1.317-360.0,624.99,2-3 Werktage,230 / 50,20 - max. 180 / 2 - max. 18,max. 600,60.0,max. 60,3.0,5.0,gelb,17.8,23.0,459 x 330 x 669,"G 180 Q Smart Control, Multi Jet 3-in-1, HD-Sc...","Integrierte HD-Schlauchtrommel, Integriertes A..."


In [9]:
project_root = Path.cwd()
test_csv     = project_root / "data" / "processed" / "SAP_Produktstammdaten_vfinal_test_missing.csv"
df_test      = pd.read_csv(test_csv, sep=",", engine="python", quotechar='"', skipinitialspace=True)

missing_locs = [
    (i, col)
    for i, row in df_test.iterrows()
    for col in df_test.columns
    if pd.isna(row[col])
]
print(f"Found {len(missing_locs)} missing cells. Sample:")
print(missing_locs[:10])

Found 134 missing cells. Sample:
[(0, 'Zulauftemperatur (°C)'), (2, 'Abmessungen (L × B × H) (mm)'), (4, 'Bestellnummer'), (4, 'Fördermenge (l/h)'), (4, 'Ausstattung'), (5, 'Druck (bar/MPa)'), (5, 'Flächenleistung (m²/h)'), (5, 'Gewicht inkl. Verpackung (kg)'), (6, 'Zulauftemperatur (°C)'), (7, 'Lieferumfang')]


Great. The display data is hard to be viewed. We need to find a way to structure the missing data in order to make the work with the LLM easyer

In [10]:
project_root = Path.cwd()
test_csv     = project_root / "data" / "processed" / "SAP_Produktstammdaten_vfinal_test_missing.csv"
df_test      = pd.read_csv(test_csv, sep=",", engine="python",
                           quotechar='"', skipinitialspace=True)

cols = [c for c in df_test.columns if c != "Produktname"]

for _, row in df_test.iterrows():
    missing = [c for c in cols if pd.isna(row[c])]
    if missing:
        prod = row["Produktname"]
        print(f"In {prod} we are missing: {', '.join(missing)}")


In K 7 Premium Smart Control Flex eco!B we are missing: Zulauftemperatur (°C)
In K 7 Premium Power Flex we are missing: Abmessungen (L × B × H) (mm)
In K 7 Premium Smart Control Flex we are missing: Bestellnummer, Fördermenge (l/h), Ausstattung
In K 7 Smart Control Flex eco!Booster we are missing: Druck (bar/MPa), Flächenleistung (m²/h), Gewicht inkl. Verpackung (kg)
In K 7 Smart Control Flex Home we are missing: Zulauftemperatur (°C)
In K 7 Power Flex we are missing: Lieferumfang
In K 7 Power Flex Home we are missing: Anschlussleistung (kW)
In K 7 Smart Control Flex we are missing: Fördermenge (l/h), Ausstattung
In K 7 WCM Premium Home we are missing: Zulauftemperatur (°C), Anschlusskabel (m)
In K 7 WCM we are missing: Fördermenge (l/h)
In K 7 WCM Car&Home we are missing: Fördermenge (l/h), Zulauftemperatur (°C)
In K 7 WCM FJ we are missing: Flächenleistung (m²/h)
In K 7 WCM Premium we are missing: Druck (bar/MPa), Lieferumfang
In K 5 Power Control Flex we are missing: Bestellnummer
I

In [11]:
project_root = Path.cwd()
csv_path     = project_root / "data" / "processed" / "SAP_Produktstammdaten_vfinal_test_missing.csv"
df_test      = pd.read_csv(csv_path, sep=",", engine="python", quotechar='"', skipinitialspace=True)

cols = [c for c in df_test.columns if c != "Produktname"]
product = None
fields  = None

for _, row in df_test.iterrows():
    missing = [c for c in cols if pd.isna(row[c])]
    if missing:
        product = row["Produktname"]
        fields  = missing
        break

print(f"Product with missing data: {product}")
print(f"Missing fields: {fields}")

Product with missing data: K 7 Premium Smart Control Flex eco!B
Missing fields: ['Zulauftemperatur (°C)']


In [22]:
from decouple import config
from dotenv import load_dotenv
from ibm_watsonx_ai import APIClient
from ibm_watsonx_ai import Credentials
from ibm_watsonx_ai.foundation_models import ModelInference
from ibm_watsonx_ai.foundation_models.schema import TextGenParameters
from tqdm import tqdm
from sklearn.metrics import classification_report


# Load .env file
load_dotenv() 

# Get the API key
WX_API_KEY = os.getenv('WX_API_KEY')
PROJECT_ID = os.getenv('WX_Project_ID')

credentials = Credentials(
    url="https://us-south.ml.cloud.ibm.com",
    api_key=WX_API_KEY
)
client = APIClient(credentials=credentials, project_id=PROJECT_ID)

In [23]:
PARAMS = TextGenParameters(
    temperature=0,
    max_new_tokens=50,
    stop_sequences=["\n"]
)
model = ModelInference(
    api_client=client,
    model_id="ibm/granite-13b-instruct-v2",
    params=PARAMS
)

In [24]:
desc_dir = project_root / "parsed_markdown"
descriptions = {}
for md in desc_dir.glob("*.md"):
    product = md.stem 
    text    = (md.read_text(encoding="utf-8"))
    descriptions[product] = text

print("Loaded descriptions for", len(descriptions), "products")

Loaded descriptions for 69 products


In [25]:
PROMPT_TEMPLATE = """
You are an expert product-data imputer.

Im giving you:

  1. The name of exactly *one* field that is missing: {col}
  2. The product name: {product}
  3. A list of all *other* fields and values in the row
  4. A full raw markdown dump

Your **only** job is to return *exactly* one thing: the *value* for {col}.
- **No** explanations, **no** punctuation around it, **no** units.
- If you cannot confidently infer it, return **NA**.

Here is the data:

Field: {col}  
Product: {product}  

Known fields:  
{other_fields}

Raw markdown:  
{markdown}
"""

# Copy your test DataFrame and locate all missing cells
filled = df_test.copy()
missing_locs = [
    (i, col)
    for i, row in df_test.iterrows()
    for col in df_test.columns
    if pd.isna(row[col])
]

# Helper to safely grab a number from the model’s text
def extract_first_number(s: str) -> float:
    m = re.search(r"-?\d+(?:[.,]\d+)?", s)
    return float(m.group(0).replace(",", ".")) if m else np.nan

# Loop through each missing cell and call the model
for (i, col) in missing_locs:
    product = filled.at[i, "Produktname"] or ""
    other_fields = "\n".join(
        f"- {c}: {filled.at[i, c]}"
        for c in filled.columns
        if c != col and pd.notna(filled.at[i, c])
    )
    raw_md = descriptions.get(product, "")

    prompt = PROMPT_TEMPLATE.format(
        col=col,
        product=product,
        other_fields=other_fields,
        markdown=raw_md
    )

    # Send the prompt
    resp = model.generate(prompt)
    raw_guess = resp["results"][0]["generated_text"].strip()

    # Fill in either as a number or as raw text/NA
    if col in numeric_cols:
        guess = extract_first_number(raw_guess)
    else:
        guess = raw_guess or "NA"

    filled.iat[i, filled.columns.get_loc(col)] = guess


In [26]:
records = []
for i, col in missing_locs:
    records.append({
        'Field':   col,
        'Imputed': filled.at[i, col],
        'Actual':  df.at[i, col]
    })

compare_df = pd.DataFrame(records, columns=['Field','Imputed','Actual'])
print(compare_df)


                             Field                       Imputed  \
0            Zulauftemperatur (°C)      1. Zulauftemperatur (°C)   
1     Abmessungen (L × B × H) (mm)  Abmessungen (L × B × H) (mm)   
2                    Bestellnummer                       1. 17.8   
3                Fördermenge (l/h)                          17.8   
4                      Ausstattung                            NA   
..                             ...                           ...   
129             Anschlusskabel (m)                           1.0   
130  Gewicht inkl. Verpackung (kg)                           1.0   
131             Anschlusskabel (m)                           1.0   
132          Preis (€ inkl. MwSt.)                           1.0   
133                Stromart (V/Hz)                         1. NA   

                                                Actual  
0                                              max. 60  
1                                      459 x 330 x 669  
2           

We can already read that the model is halucinating. Lets format the output in order to make it easyer to analyse visually

In [27]:
from IPython.display import display

# after you’ve built `compare_df`:
display(compare_df)

,Field,Imputed,Actual
0,Zulauftemperatur (°C),1. Zulauftemperatur (°C),max. 60
1,Abmessungen (L × B × H) (mm),Abmessungen (L × B × H) (mm),459 x 330 x 669
2,Bestellnummer,1. 17.8,1.317-360.0
3,Fördermenge (l/h),17.8,max. 600
4,Ausstattung,NA,"Integrierte HD-Schlauchtrommel, Integriertes A..."
...,...,...,...
129,Anschlusskabel (m),1.0,NaN
130,Gewicht inkl. Verpackung (kg),1.0,4.6
131,Anschlusskabel (m),1.0,NaN
132,Preis (€ inkl. MwSt.),1.0,NaN


In [28]:
mismatches = compare_df[
    compare_df['Imputed'].astype(str) != compare_df['Actual'].astype(str)
]
display(mismatches)


,Field,Imputed,Actual
0,Zulauftemperatur (°C),1. Zulauftemperatur (°C),max. 60
1,Abmessungen (L × B × H) (mm),Abmessungen (L × B × H) (mm),459 x 330 x 669
2,Bestellnummer,1. 17.8,1.317-360.0
3,Fördermenge (l/h),17.8,max. 600
4,Ausstattung,NA,"Integrierte HD-Schlauchtrommel, Integriertes A..."
...,...,...,...
129,Anschlusskabel (m),1.0,NaN
130,Gewicht inkl. Verpackung (kg),1.0,4.6
131,Anschlusskabel (m),1.0,NaN
132,Preis (€ inkl. MwSt.),1.0,NaN


In [29]:
error_counts = mismatches.groupby('Field').size().sort_values(ascending=False)
print(error_counts)


Field
Preis (€ inkl. MwSt.)            16
Ausstattung                      15
Anschlusskabel (m)               12
Druck (bar/MPa)                  10
Gewicht inkl. Verpackung (kg)     9
Farbe                             8
Lieferumfang                      8
Lieferzeit                        8
Fördermenge (l/h)                 7
Zulauftemperatur (°C)             7
Abmessungen (L × B × H) (mm)      6
Anschlussleistung (kW)            6
Flächenleistung (m²/h)            6
Gewicht ohne Zubehör (kg)         6
Bestellnummer                     5
Stromart (V/Hz)                   3
dtype: int64


In [31]:
n_total   = len(compare_df)
n_correct = (compare_df['Imputed'].astype(str) == compare_df['Actual'].astype(str)).sum()
print(f"Correct imputations: {n_correct}/{n_total} ({n_correct/n_total:.1%})")


Correct imputations: 2/134 (1.5%)


As we can see, no correct imput was correct. We can also see that the model returned for some of the fields the name of the category as value (e.g.: Abmessungen (L × B × H) (mm)	Abmessungen (L × B × H) (mm)). Lets change our promt in order to see if we can get rid of the halucination.

In [32]:
BASE_SYSTEM = """
You are an expert product-data imputer.
When asked for a single missing field, return exactly one token: the missing value and nothing else (no quotes, no units, no explanation). If you cannot determine it, respond with NA. Always use dot for decimals.
"""

FEW_SHOT_TEXT = """
### EXAMPLE 1
Field: Druck (bar/MPa)  (type: numeric)
Product: K 2 Battery
Markdown:
### Description
A portable battery washer …

## Technische Daten
- Druck (bar/MPa): max. 110
- Fördermenge (l/h): 340

→ 110

### EXAMPLE 2
Field: Farbe  (type: categorical)
Product: K 7 Premium Power Flex
Markdown:
### Description
Yellow-and-black pressure washer …

## Technische Daten
- Gewicht ohne Zubehör (kg): 17.8
- Gewicht inkl. Verpackung (kg): 22.2

→ gelb
"""

# 2) Copy df_test & find missing cells
filled = df_test.copy()
missing_locs = [
    (i, col)
    for i, row in df_test.iterrows()
    for col in df_test.columns
    if pd.isna(row[col])
]

# 3) Helper to extract a number
def extract_first_number(s: str) -> float:
    m = re.search(r"-?\d+(?:[.,]\d+)?", s)
    return float(m.group(0).replace(",", ".")) if m else np.nan

# 4) Loop & impute
for (i, col) in missing_locs:
    product = filled.at[i, "Produktname"] or ""
    dtype   = "numeric" if col in numeric_cols else "categorical"
    other   = "\n".join(f"- {c}: {filled.at[i,c]}" 
                        for c in filled.columns 
                        if c != col and pd.notna(filled.at[i,c]))
    md      = descriptions.get(product, "")

    # 5) Assemble one big prompt string
    prompt_text = "\n".join([
        BASE_SYSTEM,
        FEW_SHOT_TEXT,
        f"### YOUR TURN",
        f"Field: {col}  (type: {dtype})",
        f"Product: {product}",
        "Markdown:",
        md,
        "Other fields:",
        other,
        "→"
    ])

    # 6) Call the model with a single string
    resp = model.generate(prompt=prompt_text)
    raw = resp["results"][0]["generated_text"].strip()

    # 7) Coerce numeric vs categorical
    if col in numeric_cols:
        guess = extract_first_number(raw)
    else:
        guess = raw or "NA"

    filled.iat[i, filled.columns.get_loc(col)] = guess

In [33]:
records = []
for i, col in missing_locs:
    records.append({
        'Field':   col,
        'Imputed': filled.at[i, col],
        'Actual':  df.at[i, col]
    })

compare_df = pd.DataFrame(records, columns=['Field','Imputed','Actual'])
print(compare_df)


                             Field                         Imputed  \
0            Zulauftemperatur (°C)                              29   
1     Abmessungen (L × B × H) (mm)                   180 x 80 x 80   
2                    Bestellnummer  K 7 Premium Smart Control Flex   
3                Fördermenge (l/h)                             340   
4                      Ausstattung                  Andere Angaben   
..                             ...                             ...   
129             Anschlusskabel (m)                             1.4   
130  Gewicht inkl. Verpackung (kg)                            22.2   
131             Anschlusskabel (m)                             NaN   
132          Preis (€ inkl. MwSt.)                             1.6   
133                Stromart (V/Hz)                 Vario Power Jet   

                                                Actual  
0                                              max. 60  
1                                      459 x 

In [34]:
display(compare_df)

,Field,Imputed,Actual
0,Zulauftemperatur (°C),29,max. 60
1,Abmessungen (L × B × H) (mm),180 x 80 x 80,459 x 330 x 669
2,Bestellnummer,K 7 Premium Smart Control Flex,1.317-360.0
3,Fördermenge (l/h),340,max. 600
4,Ausstattung,Andere Angaben,"Integrierte HD-Schlauchtrommel, Integriertes A..."
...,...,...,...
129,Anschlusskabel (m),1.4,NaN
130,Gewicht inkl. Verpackung (kg),22.2,4.6
131,Anschlusskabel (m),NaN,NaN
132,Preis (€ inkl. MwSt.),1.6,NaN


In [35]:
mismatches = compare_df[
    compare_df['Imputed'].astype(str) != compare_df['Actual'].astype(str)
]
display(mismatches)

,Field,Imputed,Actual
0,Zulauftemperatur (°C),29,max. 60
1,Abmessungen (L × B × H) (mm),180 x 80 x 80,459 x 330 x 669
2,Bestellnummer,K 7 Premium Smart Control Flex,1.317-360.0
3,Fördermenge (l/h),340,max. 600
4,Ausstattung,Andere Angaben,"Integrierte HD-Schlauchtrommel, Integriertes A..."
...,...,...,...
128,Druck (bar/MPa),220,110 / 11
129,Anschlusskabel (m),1.4,NaN
130,Gewicht inkl. Verpackung (kg),22.2,4.6
132,Preis (€ inkl. MwSt.),1.6,NaN


The model is obviously halucinating. Lets be deterministic and try to find the value only for the first missing cell.

In [36]:
project_root = Path.cwd()
csv_path     = project_root / "data/processed/SAP_Produktstammdaten_vfinal_test_missing.csv"
clean_dir    = project_root / "parsed_markdown"

df = pd.read_csv(csv_path, sep=",", engine="python", quotechar='"', skipinitialspace=True)
cols = [c for c in df.columns if c != "Produktname"]

product = field = None
for _, row in df.iterrows():
    missing = [c for c in cols if pd.isna(row[c])]
    if missing:
        product = row["Produktname"]
        field   = missing[0]
        break

assert product and field, "No missing field found!"

md_path = clean_dir / f"{product}.md"
md_text = md_path.read_text(encoding="utf-8") if md_path.exists() else ""

BASE_SYSTEM = """
You are an expert product-data imputer.
When asked for a single missing field, you will:
1. Read the cleaned Markdown file named <Produktname>.md from parsed_markdown_clean.
2. Find the requested field.
3. Return exactly one line:
In <Produktname> the correct <Field> is <Value>
If the field is absent, return:
In <Produktname> the correct <Field> is NaN
Use a dot for decimals; no quotes, units, or extra text.
"""

FEW_SHOT = """
### EXAMPLE 1
Field: Druck (bar/MPa)
Product: K 2 Battery
(Excerpt:)
- Druck (bar/MPa): max. 110
→ In K 2 Battery the correct Druck (bar/MPa) is 110

### EXAMPLE 2
Field: Farbe
Product: K 7 Premium Power Flex
(Excerpt:)
- Gewicht ohne Zubehör (kg): 17.8
→ In K 7 Premium Power Flex the correct Farbe is NaN
"""

prompt = (
    BASE_SYSTEM.strip() + "\n\n" +
    FEW_SHOT.strip() + "\n\n" +
    f"Field: {field}\n" +
    f"Product: {product}\n\n" +
    f"(Contents of {product}.md below:)\n" +
    md_text + "\n\n" +
    "→"
)

In [37]:
result_line = model.generate_text(prompt)
print(result_line.strip())

In K 7 Premium Smart Control Flex eco!B the correct Zulauftemperatur (°C) is 20


As we can see, the model is halucinating even with one data imput. The actual value is **Zulauftemperatur (°C)**: max. 60. What if we try to ask the model in a promt? To reduce the possibility of an error, we use here the identical papameters and code that we used in MA3 from our group.

In [38]:
from langchain.llms import WatsonxLLM
from langchain_ibm import WatsonxLLM

llm = WatsonxLLM(
    model_id="ibm/granite-13b-instruct-v2",
    url="https://us-south.ml.cloud.ibm.com",
    apikey=WX_API_KEY,
    project_id=PROJECT_ID,
    params={
        "decoding_method": "greedy",
        "temperature": 0.0,
        "min_new_tokens": 5,
        "max_new_tokens": 1_000,
        "repetition_penalty": 1.2,
    },
)


In [39]:
llm_result = llm.invoke("Hi how are you?")

print(type(llm_result))
print(llm_result)

<class 'str'>
I'm good. How about you? 


In [40]:
llm_result = llm.invoke(
    "You are a product-data extractor. In the text below, locate and return the value for “Zulauftemperatur (°C)”. If the field isn’t present, respond with “Not found”.\n\n"
    "Text:\n\n"
    "## Page Header\n\n"
    "The image shows the logo of Kärcher, a company known for its cleaning equipment. The logo consists of the word \"KÄRCHER\" in bold, black uppercase letters. Below the text, there is a yellow horizontal bar. The background is white, providing contrast to the black text and yellow bar. <!-- page_header, ID 77001531-ac4c-4427-8d9d-4871b3fbf695 -->\n\n"
    "# K 5 PREMIUM SMART CONTROL FLEX eco!Booster <!-- title, ID 18485f43-17b0-4038-a9bf-5ed087e286ad -->\n\n"
    "## Text\n\n"
    "Für mehr Performance: der Hochdruckreiniger K 5 Premium Smart Control Flex eco!Booster mit PremiumFlex-Schlauch, G 180 Q Smart Control-Pistole, Schlauchtrommel und eco!Booster Kit. <!-- text, ID 743bca50-0296-4a44-872f-f6134441c566 -->\n\n"
    "## Description\n\n"
    "The image displays a Kärcher pressure washer, specifically the K5 Smart Control model. The device is predominantly yellow with black accents and features a sleek, modern design. It includes a handle for easy maneuverability and wheels for transport. The pressure washer is equipped with a hose reel for convenient storage of the hose.\n\n"
    "### Components\n\n"
    "- **Pressure Washer Unit**:\n"
    "  - **Color**: Yellow and black.\n"
    "  - **Model**: K5 Smart Control.\n"
    "  - **Brand**: Kärcher.\n"
    "  - **Features**:\n"
    "    - Integrated handle.\n"
    "    - Wheels for mobility.\n"
    "    - Hose reel for storage.\n\n"
    "- **Accessories**:\n"
    "  - **Spray Gun**:\n"
    "    - Black with yellow accents.\n"
    "    - Branded with Kärcher logo.\n"
    "  - **Lance**:\n"
    "    - Black with yellow tip.\n"
    "    - Designed for various cleaning tasks.\n"
    "  - **Detergent Bottle**:\n"
    "    - Labeled \"Universal\".\n"
    "    - Features an image of a person using the pressure washer.\n\n"
    "### Additional Details\n\n"
    "- The pressure washer is designed for home use, suitable for cleaning cars, patios, and other surfaces.\n"
    "- The image suggests a focus on versatility and ease of use, with the inclusion of multiple attachments and a detergent bottle for enhanced cleaning performance. <!-- figure, ID dac59e3f-1a35-4150-a02f-2cf0e71df6cb -->\n\n"
    "## Price\n\n"
    "The image shows a price of 539,99 €. <!-- text, ID 27d5d7da-d92d-4c2d-b800-17bbfad4ad1a -->\n\n"
    "## Text Content\n\n"
    "inkl. MwSt. - kostenlose Lieferung ab 50 € <!-- text, ID 1a5b0407-8076-462f-9b79-cc46a8d4e8f2 -->\n\n"
    "## Page Footer\n\n"
    "[https://www.kaercher.com/de/home-garden/hochdruckreiniger/k-5-premium-smart-control-flex-eco-booster-13246870.html](https://www.kaercher.com/de/home-garden/hochdruckreiniger/k-5-premium-smart-control-flex-eco-booster-13246870.html) <!-- page_footer, ID 7567dd32-5f11-4cda-a5d8-5011045f6c0d -->\n\n"
    "## Page Number\n\n"
    "1/10 <!-- page_number, ID c25e9df6-69c3-4714-8eb0-9b4517817945 -->\n\n"
    "## Page Header\n\n"
    "30/04/2025, 23:44 <!-- page_header, ID 200ff78b-640d-4cb1-8ef9-6c5883e4ef6d -->\n\n"
    "## K 5 Premium Smart Control Flex eco!Booster | Kärcher\n\n"
    "This is a page header indicating the title of a product from Kärcher, specifically the \"K 5 Premium Smart Control Flex eco!Booster.\" <!-- page_header, ID e51076b1-3049-4369-894d-be3c7988960f -->\n\n"
    "## Text Information\n\n"
    "- Lieferbar in 3-4 Werktagen\n"
    "- Bestellnummer: 1.324-687.0 <!-- key_value, ID 4be821ac-2c5f-4ae2-8a20-8924b72d464c -->\n\n"
    "## Händler Suche\n\n"
    "- **Ort oder PLZ**: [Text input field]\n\n"
    "### Bewertung\n\n"
    "- **Sterne**: [ ] [ ] [ ] [ ] [ ] (0)\n"
    "- **Aktion**: Jetzt Produkt bewerten\n\n"
    "### Produkt Optionen\n\n"
    "- **Produkt vergleichen**: [Text link]\n\n"
    "### Hilfe\n\n"
    "- **Benötigen Sie Hilfe?**\n"
    "  - **Hotline**: +49 7195 903 0 <!-- form, ID 50cec018-d8bb-4273-9abc-1bad6bea78f9 -->\n\n"
    "## Text Content\n\n"
    "Dank integriertem Bluetooth lässt sich der Hochdruckreiniger K 5 Premium Smart Control Flex eco!Booster mit der Kärcher Home & Garden App verbinden. Die App bietet viele nützliche Funktionen wie den Anwendungsberater mit hilfreichen Tipps und Tricks, eine Aufbauanleitung, Wartungs- und Pflegehinweise sowie das Kärcher Serviceportal. Darüber hinaus verfügt das Gerät über einen Boost Mode für extra Power, G 180 Q Smart Control-Pistole mit LCD-Display und das 3-in-1-Multi Jet-Strahlrohr. Die Druckeinstellungen werden direkt an der Pistole vorgenommen oder mit Hilfe des Anwendungsberaters aus der App auf die Pistole übertragen. Über das LCD-Display lässt sich überprüfen, welche Druckstufe eingestellt ist. Weitere Ausstattungsdetails sind die Schlauchtrommel, das Plug 'n' Clean-Reinigungssystem, der PremiumFlex-Hochdruckschlauch, der Aluminium-Teleskopgriff sowie die Parkposition für jederzeit griffbereites Zubehör. Inklusive eco!Booster Kit mit eco!Booster und 1 Liter Universalreiniger. Der eco!Booster ist ideal für empfindliche Oberflächen und sorgt mit einer um 50 Prozent höheren Reinigungsleistung im Vergleich zum Flachstrahl für eine Wasser-, Energie- und Zeitersparnis. <!-- text, ID e6a43fc4-0308-4ec7-8aa5-064878973f75 -->\n\n"
    "## Merkmale und Vorteile <!-- text, ID 8f4cbbb5-c478-495f-a400-8e93008927ce -->\n\n"
    "## Page Number\n\n"
    "5/10 <!-- page_number, ID 8a12ffa8-af64-4455-a64a-da610b8122d2 -->\n\n"
    "### Page Header\n\n"
    "- **Date and Time**: 30/04/2025, 23:44 <!-- page_header, ID 2988337e-2b21-44bd-9e82-ce3230efd6c9 -->\n\n"
    "## K 5 Premium Smart Control Flex eco!Booster | Kärcher <!-- page_header, ID f648a450-9186-4323-b735-28539ff3487d -->\n\n"
    "## Schlauchtrommel für komfortable Handhabung\n\n"
    "- Der Hochdruckschlauch ist optimal geschützt und platzsparend verstaut.\n"
    "- Bequemes Arbeiten: Jederzeit griffbereiter Schlauch durch leichtes Auf- und Abrollen.\n"
    "- Tiefer Schwerpunkt für einen sicheren Stand auch auf schrägen Oberflächen. <!-- text, ID 7796f8d0-5d1d-4fb7-b8f3-7f4049ce99ed -->\n\n"
    "## Wahrgenommene Lautstärkenreduktion\n\n"
    "- Angenehmer Klang der Anwendung**\n\n"
    "** Vergleich zur wahrgenommenen Lautstärke bei der Anwendung des Kärcher Standard-Flachstrahls. <!-- text, ID 16b63c93-743d-4bb8-a0a2-500169bdbed3 -->\n\n"
    "## SPEZIFIKATIONEN\n\n"
    "## Technische Daten\n\n"
    "- Stromart (V/Hz): 230 / 50\n"
    "- Druck (bar/MPa): 20 - max. 145 / 2 - max. 14,5\n"
    "- Fördermenge (l/h): max. 500\n"
    "- Flächenleistung (m²/h): 40\n"
    "- Zulauftemperatur (°C): max. 60\n"
    "- Anschlussleistung (kW): 2,1\n"
    "- Anschlusskabel (m): 5\n"
    "- Farbe: gelb\n"
    "- Gewicht ohne Zubehör (kg): 13,5\n"
    "- Gewicht inkl. Verpackung (kg): 18,5\n"
    "- Abmessungen (L × B × H) (mm): 417 × 306 × 584 <!-- key_value, ID ff4355bf-a76b-40ac-a30b-21aa7848d77b -->\n\n"
    "## Lieferumfang\n\n"
    "- Hochdruckpistole: G 180 Q Smart Control\n"
    "- Multi Jet 3-in-1\n"
    "- eco!Booster <!-- text, ID 2050473b-7124-4cbb-8fda-9aaaf6e6f335 -->\n\n"
    "## Page Footer\n\n"
    "[https://www.kaercher.com/de/home-garden/hochdruckreiniger/k-5-premium-smart-control-flex-eco-booster-13246870.html](https://www.kaercher.com/de/home-garden/hochdruckreiniger/k-5-premium-smart-control-flex-eco-booster-13246870.html) <!-- page_footer, ID 29726f76-d2b3-4d03-b2b5-b77372b17c05 -->\n\n"
    "### Page Number\n\n"
    "6/10 <!-- page_number, ID 0b62e4cc-caff-40a0-b2b7-4b438d3c250e -->"
)
print(llm_result)



Output: Not found


As we can see, the model is halucinating even with direct data imput in the promt. The actual value is **Zulauftemperatur (°C)**: max. 60. What if we clean the markdowns and try to leave only the "Technical Details" like temperature?

In [41]:
project_root = Path.cwd()
src_dir      = project_root / "parsed_markdown"
clean_dir    = project_root / "parsed_markdown_clean"

clean_dir.mkdir(exist_ok=True)

In [42]:
comment_re          = re.compile(r"<!--.*?-->", flags=re.DOTALL)
blank_re            = re.compile(r"(?:\r?\n){3,}")
remove_sections_re  = re.compile(
    r"^##\s+(?:Page Header|Page Footer|Page Number)[\s\S]*?(?=^##\s|\Z)",
    flags=re.MULTILINE,
)

KEYS = [
    "Bestellnummer",
    "Preis",                 
    "Lieferzeit",
    "Stromart",
    "Druck",
    "Fördermenge",
    "Flächenleistung",
    "Zulauftemperatur",
    "Anschlussleistung",
    "Anschlusskabel",
    "Farbe",
    "Gewicht ohne Zubehör",
    "Gewicht inkl. Verpackung",
    "Abmessungen",
]


key_line_re = re.compile(
    rf"^\s*(?:[-*]\s*)?(?:\*\*|__)?(?:{'|'.join(map(re.escape, KEYS))})(?:\b|:)",
    flags=re.IGNORECASE,
)


SECTION_TITLES = {"Lieferumfang", "Ausstattung"}

for md_path in src_dir.glob("*.md"):
    raw = md_path.read_text(encoding="utf-8")


    txt = remove_sections_re.sub("", comment_re.sub("", raw))

    out_lines        = []
    keep_section     = False      
    product_written  = False      # first non-page heading becomes Produktname

    for line in txt.splitlines():

        if line.startswith("## "):
            hdr = line.lstrip("#").strip()


            if not product_written:
                prod_name = hdr.split("|")[0].strip()
                out_lines.append("### Produktname")
                out_lines.append(prod_name)
                product_written = True
                keep_section = False
                continue


            if hdr in SECTION_TITLES:
                out_lines.append(f"### {hdr}")
                keep_section = True
            else:
                keep_section = False

            continue  

        if keep_section or key_line_re.match(line):
            out_lines.append(line)

    cleaned = blank_re.sub("\n\n", "\n".join(out_lines).strip()) + "\n"
    (clean_dir / md_path.name).write_text(cleaned, encoding="utf-8")

print(
    f"Cleaned markdown saved for {len(list(src_dir.glob('*.md')))} files into {clean_dir}"
)

Cleaned markdown saved for 69 files into c:\Users\charl\Documents\CBS\Code Projects_Python\AIML25\AIML25_Project\parsed_markdown_clean


In [43]:
desc_dir = project_root / "parsed_markdown_clean"
descriptions = {}
for md in desc_dir.glob("*.md"):
    product = md.stem 
    text    = (md.read_text(encoding="utf-8"))
    descriptions[product] = text

print("Loaded descriptions for", len(descriptions), "products")

Loaded descriptions for 69 products


In [44]:
PROMPT_TEMPLATE = """
You are an expert product-data imputer.

Im giving you:

  1. The name of exactly *one* field that is missing: {col}
  2. The product name: {product}
  3. A list of all *other* fields and values in the row
  4. A full raw markdown dump

Your **only** job is to return *exactly* one thing: the *value* for {col}.
- **No** explanations, **no** punctuation around it, **no** units.
- If you cannot confidently infer it, return **NA**.

Here is the data:

Field: {col}  
Product: {product}  

Known fields:  
{other_fields}

Raw markdown:  
{markdown}
"""

# Copy your test DataFrame and locate all missing cells
filled = df_test.copy()
missing_locs = [
    (i, col)
    for i, row in df_test.iterrows()
    for col in df_test.columns
    if pd.isna(row[col])
]

# Helper to safely grab a number from the model’s text
def extract_first_number(s: str) -> float:
    m = re.search(r"-?\d+(?:[.,]\d+)?", s)
    return float(m.group(0).replace(",", ".")) if m else np.nan

# Loop through each missing cell and call the model
for (i, col) in missing_locs:
    product = filled.at[i, "Produktname"] or ""
    other_fields = "\n".join(
        f"- {c}: {filled.at[i, c]}"
        for c in filled.columns
        if c != col and pd.notna(filled.at[i, c])
    )
    raw_md = descriptions.get(product, "")

    prompt = PROMPT_TEMPLATE.format(
        col=col,
        product=product,
        other_fields=other_fields,
        markdown=raw_md
    )

    # Send the prompt
    resp = model.generate(prompt)
    raw_guess = resp["results"][0]["generated_text"].strip()

    # Fill in either as a number or as raw text/NA
    if col in numeric_cols:
        guess = extract_first_number(raw_guess)
    else:
        guess = raw_guess or "NA"

    filled.iat[i, filled.columns.get_loc(col)] = guess


In [45]:
records = []
for i, col in missing_locs:
    records.append({
    'Field':   col,
    'Imputed': filled.at[i, col],
    'Actual':  original.at[i, col]  # ← now you grab from your unmasked copy
})


compare_df = pd.DataFrame(records, columns=['Field','Imputed','Actual'])
print(compare_df)


                             Field                       Imputed  \
0            Zulauftemperatur (°C)      1. Zulauftemperatur (°C)   
1     Abmessungen (L × B × H) (mm)  Abmessungen (L × B × H) (mm)   
2                    Bestellnummer                       1. 17.8   
3                Fördermenge (l/h)                          17.8   
4                      Ausstattung                            NA   
..                             ...                           ...   
129             Anschlusskabel (m)                           1.0   
130  Gewicht inkl. Verpackung (kg)                           1.0   
131             Anschlusskabel (m)                           1.0   
132          Preis (€ inkl. MwSt.)                           1.0   
133                Stromart (V/Hz)                         1. NA   

                                                Actual  
0                                              max. 60  
1                                      459 x 330 x 669  
2           

In [46]:
display(compare_df)

,Field,Imputed,Actual
0,Zulauftemperatur (°C),1. Zulauftemperatur (°C),max. 60
1,Abmessungen (L × B × H) (mm),Abmessungen (L × B × H) (mm),459 x 330 x 669
2,Bestellnummer,1. 17.8,1.317-360.0
3,Fördermenge (l/h),17.8,max. 600
4,Ausstattung,NA,"Integrierte HD-Schlauchtrommel, Integriertes A..."
...,...,...,...
129,Anschlusskabel (m),1.0,nicht angegeben
130,Gewicht inkl. Verpackung (kg),1.0,4.6
131,Anschlusskabel (m),1.0,nicht angegeben
132,Preis (€ inkl. MwSt.),1.0,nicht verfügbar


In [47]:
error_counts = mismatches.groupby('Field').size().sort_values(ascending=False)
print(error_counts)

Field
Preis (€ inkl. MwSt.)            16
Ausstattung                      15
Anschlusskabel (m)               11
Druck (bar/MPa)                  10
Gewicht inkl. Verpackung (kg)     9
Lieferumfang                      8
Lieferzeit                        8
Anschlussleistung (kW)            7
Flächenleistung (m²/h)            7
Fördermenge (l/h)                 7
Zulauftemperatur (°C)             7
Abmessungen (L × B × H) (mm)      6
Gewicht ohne Zubehör (kg)         6
Bestellnummer                     5
Stromart (V/Hz)                   3
dtype: int64


Seems no luck for now. But can it actually extract values from a "curated" imput?

In [48]:
llm_result = llm.invoke(
    "You are a product-data extractor. In the text below, locate and return the value for “Zulauftemperatur (°C)”. " 
    "If the field isn’t present, respond with “Not found”.\n\n"
    "Text:\n"
    "### Technische Daten\n"
    "- Stromart (Ph/V/Hz): 1 / 230 / 50\n"
    "- Druck (bar/MPa): 20 - max. 180 / 2 - max. 18\n"
    "- Fördermenge (l/h): max. 600\n"
    "- Flächenleistung (m²/h): 60\n"
    "- Zulauftemperatur (°C): max. 60\n"
    "- Anschlussleistung (kW): 3\n"
    "- Anschlusskabel (m): 5\n"
    "- Farbe: gelb\n"
    "- Gewicht ohne Zubehör (kg): 17,3\n"
    "- Gewicht inkl. Verpackung (kg): 24\n"
    "- Abmessungen (L × B × H) (mm): 458 × 330 × 669"
)
print(llm_result)



max. 60




Great so our 13 Billion parameter model is working, but is to small to perform tasks designated for a quality check tool.